# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1467, -0.1259, -0.3079, -0.1921, -0.2227,  0.1404,  0.0484, -0.2500,
         -0.1126,  0.1275],
        [-0.1008, -0.1865, -0.2723, -0.1512, -0.1974,  0.1182, -0.0776, -0.0249,
         -0.2117,  0.2002]], grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.2587,  0.0077, -0.0152, -0.0801,  0.0909,  0.0766, -0.3063,  0.1413,
          0.2121, -0.0381],
        [ 0.0899,  0.0496, -0.0506,  0.0304,  0.1530,  0.0693, -0.1982,  0.1892,
          0.0809, -0.0129]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0820, -0.0776, -0.0659, -0.0075, -0.1185, -0.0005,  0.1571, -0.1571,
         -0.0343, -0.3000],
        [ 0.1122, -0.0706, -0.0532,  0.0521, -0.2315,  0.0262, -0.0751,  0.0285,
         -0.1170, -0.1475]], grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0919, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.4179, grad_fn=<SumBackward0>)